In [ ]:
import cv2
import yt_dlp
import base64
import httpx
import logging
import asyncio
import urllib.parse
import io
import uuid
from src.core.config import settings
from src.core.database import SessionLocal
from src.models.video import Video
from src.core.storage import storage_client
from src.core.kafka import kafka_producer
import edge_tts

logger = logging.getLogger("worker.analysis")

class VideoAnalysisService:
    def __init__(self, video_id: int):
        self.video_id = video_id

    def _get_db(self):
        return SessionLocal()

    def update_status(self, status: str, summary: str = None, thumbnail_url: str = None, audio_url: str = None) -> str:
        """
        DB 상태 및 결과 업데이트 (통합 메서드)
        반환값: 알림 전송에 사용할 비디오 제목 (title)
        """
        db = self._get_db()
        video_title = "Unknown Video"
        try:
            video = db.query(Video).filter(Video.id == self.video_id).first()
            if video:
                video.analysis_status = status
                if summary:
                    video.ai_summary = summary
                
                # 썸네일 업데이트 (Worker가 생성한 경우 AI 썸네일로 표시)
                if thumbnail_url:
                    video.thumbnail = thumbnail_url
                    video.is_ai_thumbnail = True
                
                # 오디오 URL 업데이트
                if audio_url:
                    video.ai_audio = audio_url

                db.commit()
                db.refresh(video)
                video_title = video.title
                logger.info(f"Video {self.video_id} status updated to '{status}'")
        except Exception as e:
            db.rollback()
            logger.error(f"Failed to update status to {status}: {e}")
        finally:
            db.close()
        
        return video_title

    async def finalize_analysis(self, status: str, summary: str = None, thumbnail_url: str = None, audio_url: str = None):
        """
        분석 종료 처리 (DB 업데이트 및 알림 전송)
        성공(completed)과 실패(failed) 상황을 모두 처리합니다.
        """
        
        # 1. DB Update (상태에 따라 업데이트)
        video_title = self.update_status(status, summary, thumbnail_url, audio_url)

        # 2. Notification Event (Kafka)
        try:
            notification_message = {
                "event": f"analysis_{status}", # analysis_completed 또는 analysis_failed
                "video_id": self.video_id,
                "title": video_title,
                "status": status, # [수정] 하드코딩 제거 (변수 사용)
                "summary_preview": summary[:100] + "..." if summary else "No summary available"
            }
            
            topic = getattr(settings, "KAFKA_TOPIC_NOTIFICATION", "video-notifications")
            await kafka_producer.send_message(topic, notification_message)
            logger.info(f"Sent notification event for Video {self.video_id} (Status: {status})")
        except Exception as e:
            logger.error(f"Failed to send notification event: {e}")

    def get_stream_url(self, youtube_url: str) -> str:
        ydl_opts = {
            'format': 'best[ext=mp4]', 
            'quiet': True,
            'extractor_args': {'youtube': {'player_client': ['android', 'web']}}
        }
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            info = ydl.extract_info(youtube_url, download=False)
            return info['url']

    def extract_keyframes(self, video_url: str, interval_sec: int = 5) -> list:
        frames = []
        try:
            target_url = self.get_stream_url(video_url) if "youtu" in video_url else video_url
            cap = cv2.VideoCapture(target_url)
            fps = cap.get(cv2.CAP_PROP_FPS) or 30
            interval = int(fps * interval_sec)
            count = 0
            while cap.isOpened():
                ret, frame = cap.read()
                if not ret or len(frames) >= 10: break
                if count % interval == 0:
                    _, buf = cv2.imencode('.jpg', frame)
                    frames.append(base64.b64encode(buf).decode('utf-8'))
                count += 1
            cap.release()
            return frames
        except Exception as e:
            logger.error(f"Frame extraction failed: {e}")
            raise e

    async def generate_image_description(self, base64_image: str) -> str:
        try:
            # [수정] verify=False 제거 (기본값 True 사용)
            async with httpx.AsyncClient(timeout=60.0) as client:
                resp = await client.post(
                    f"{settings.OLLAMA_URL}/api/generate",
                    json={
                        "model": "minicpm-v:8b", 
                        "prompt": "Describe this image in detail.",
                        "images": [base64_image],
                        "stream": False
                    }
                )
                return resp.json().get("response", "") if resp.status_code == 200 else ""
        except Exception as e:
            logger.error(f"VLM Connection Error: {e}")
            return ""

    async def summarize_descriptions(self, descriptions: list) -> str:
        text = "\n".join(descriptions)
        try:
            # [수정] verify=False 제거
            async with httpx.AsyncClient(timeout=120.0) as client:
                resp = await client.post(
                    f"{settings.OLLAMA_URL}/api/generate",
                    json={
                        "model": "qwen3:latest", 
                        "prompt": f"Summarize the following scene descriptions into a cohesive video summary in Korean:\n{text}", 
                        "stream": False
                    }
                )
                return resp.json().get("response", "") if resp.status_code == 200 else "Failed"
        except Exception as e:
            logger.error(f"LLM Connection Error: {e}")
            return "Failed"

    async def generate_thumbnail_prompt(self, summary: str) -> str:
        try:
            prompt_instruction = (
                f"Based on the summary below, write a high-quality text-to-image prompt "
                f"for a movie poster style thumbnail. Write ONLY the English prompt.\n\nSummary: {summary}"
            )
            # [수정] verify=False 제거
            async with httpx.AsyncClient(timeout=60.0) as client:
                resp = await client.post(
                    f"{settings.OLLAMA_URL}/api/generate",
                    json={
                        "model": "exaone3.5:latest",
                        "prompt": prompt_instruction,
                        "stream": False,
                        "options": {"temperature": 0.7}
                    }
                )
                return resp.json().get("response", "").strip() if resp.status_code == 200 else ""
        except Exception as e:
            logger.error(f"Prompt gen error: {e}")
            return ""

    async def generate_and_upload_thumbnail(self, summary: str) -> str:
        prompt = await self.generate_thumbnail_prompt(summary)
        if not prompt: return None

        # 프롬프트 정제
        clean_prompt = prompt.replace("**", "").replace('"', "").strip()

        try:
            encoded_prompt = urllib.parse.quote(f"cinematic shot, masterpiece, {clean_prompt}")
            # 1. Flux 모델 시도
            image_url = f"https://image.pollinations.ai/prompt/{encoded_prompt}?width=1280&height=720&model=flux&seed=42"
            
            # [수정] verify=False 제거
            async with httpx.AsyncClient(timeout=45.0) as client:
                resp = await client.get(image_url)
                
                # Flux 실패 시 Turbo 모델 폴백
                if resp.status_code != 200:
                    logger.warning(f"Flux model failed. Retrying with Turbo...")
                    image_url = f"https://image.pollinations.ai/prompt/{encoded_prompt}?width=1280&height=720&model=turbo&seed=42"
                    resp = await client.get(image_url)

                if resp.status_code != 200: return None
                image_data = io.BytesIO(resp.content)

            filename = f"ai_thumb_{self.video_id}_{uuid.uuid4().hex[:8]}.jpg"
            return storage_client.upload_file(image_data, object_name=filename)
        except Exception as e:
            logger.error(f"Thumbnail error: {e}")
            return None

    async def generate_and_upload_audio(self, text: str) -> str:
        if not text:
            return None

        try:
            communicate = edge_tts.Communicate(text, "ko-KR-SunHiNeural")
            audio_buffer = io.BytesIO()
            async for chunk in communicate.stream():
                if chunk["type"] == "audio":
                    audio_buffer.write(chunk["data"])
            
            audio_buffer.seek(0)

            filename = f"ai_audio_{self.video_id}_{uuid.uuid4().hex[:8]}.mp3"
            s3_url = storage_client.upload_file(
                audio_buffer, 
                object_name=filename, 
                content_type="audio/mpeg"
            )
            
            logger.info(f"TTS Audio generated in memory: {s3_url}")
            return s3_url

        except Exception as e:
            logger.error(f"TTS generation/upload failed: {e}")
            return None

    async def process_video(self):
        self.update_status("processing")
        
        db = self._get_db()
        url = db.query(Video.url).filter(Video.id == self.video_id).scalar()
        db.close()

        if not url:
            # 실패 시에도 알림 및 상태 업데이트를 위해 finalize_analysis 호출
            await self.finalize_analysis("failed")
            return

        try:
            # 1. 프레임 추출
            frames = self.extract_keyframes(url)
            if not frames: raise Exception("No frames extracted")

            # 2. 이미지 설명 생성
            descs = []
            for frame in frames:
                d = await self.generate_image_description(frame)
                if d: descs.append(d)
            
            # 설명 생성 실패 시 조기 실패 처리
            if not descs:
                raise Exception("Failed to generate image descriptions")

            # 3. 요약 생성
            summary = await self.summarize_descriptions(descs)
            if not summary or summary == "Failed": raise Exception("Summary generation failed")

            # 4. 썸네일 & 오디오 생성 (병렬)
            ai_thumb_task = self.generate_and_upload_thumbnail(summary)
            ai_audio_task = self.generate_and_upload_audio(summary)

            ai_thumb_url, ai_audio_url = await asyncio.gather(ai_thumb_task, ai_audio_task)

            # 5. 최종 완료 처리 (성공)
            await self.finalize_analysis("completed", summary, ai_thumb_url, ai_audio_url)
            
        except Exception as e:
            logger.error(f"Analysis failed: {e}")
            # [수정] 예외 발생 시 'failed' 상태로 알림 발행 및 DB 업데이트
            await self.finalize_analysis("failed")

In [2]:
import asyncio
import base64
import io
import logging
import urllib.parse
import uuid
import logging
import cv2
import httpx
import yt_dlp

ModuleNotFoundError: No module named 'yt_dlp'

In [7]:
logger = logging.getLogger("worker.analysis")

In [8]:
def get_stream_url(youtube_url: str) -> str:
    # [수정] JS 런타임 오류 방지를 위한 extractor_args 추가
    ydl_opts = {
        "format": "best[ext=mp4]",
        "quiet": True,
        "extractor_args": {
            "youtube": {
                "player_client": ["android", "web"]  # 모바일 클라이언트 흉내로 JS 우회 시도
            }
        },
    }
    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            info = ydl.extract_info(youtube_url, download=False)
            return info["url"]
    except Exception as e:
        logger.error(f"yt-dlp failed: {e}")
        return youtube_url
def extract_keyframes(video_url: str, interval_sec: int = 5) -> list:
    frames = []
    try:
        if "youtube.com" in video_url or "youtu.be" in video_url:
            target_url = get_stream_url(video_url)
        else:
            target_url = video_url

        cap = cv2.VideoCapture(target_url)
        fps = cap.get(cv2.CAP_PROP_FPS)
        if fps == 0:
            fps = 30  # Fallback
        frame_interval = int(fps * interval_sec)
        count = 0

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            if count % frame_interval == 0:
                _, buffer = cv2.imencode(".jpg", frame)
                b64_image = base64.b64encode(buffer).decode("utf-8")
                frames.append(b64_image)
                if len(frames) >= 15:
                    break
            count += 1
        cap.release()
        return frames
    except Exception as e:
        logger.error(f"Frame extraction error: {e}")
        raise e

In [9]:
ee = extract_keyframes("https://www.youtube.com/watch?v=sVsb9LoNJX4")

yt-dlp failed: name 'yt_dlp' is not defined


In [ ]:
prompt = "Describe this image in detail."
async with httpx.AsyncClient(timeout=60.0, verify=False) as client:
                resp = await client.post(
                    "https://ollama.hy-home.local/api/generate",
                    json={
                        "model": "qwen3-vl:8b",
                        "prompt": prompt,
                        "images": [base64_image],
                        "stream": False,
                    },
                )

In [ ]:
import React, { useState, useEffect, useMemo } from 'react';
import { create } from 'zustand';
import { persist, createJSONStorage } from 'zustand/middleware';
import { 
  Play, User, Lock, LogOut, Plus, Trash2, 
  LayoutGrid, List, Search, X, Loader2,
  FileText, Sparkles, Youtube, CheckCircle2, AlertCircle, RefreshCw
} from 'lucide-react';

/**
 * --------------------------------------------------------------------------
 * [CONFIG] 환경 설정 & Mock Data
 * --------------------------------------------------------------------------
 */
const CONFIG = {
  API_URL: 'http://localhost:8000/api/v1',
  USE_REAL_API: true,
};

const MOCK_VIDEOS = [
  { 
    id: 101, 
    title: 'Smart City Vision (Demo)', 
    category: 'Commercial', 
    thumbnail: 'https://images.unsplash.com/photo-1480714378408-67cf0d13bc1b?w=800&q=80', 
    url: 'https://www.youtube.com/watch?v=ysz5S6PUM-U', 
    description: '서버 연결이 되지 않아 표시되는 예시 데이터입니다.', 
    analysis_status: 'completed', 
    ai_summary: '이 영상은 스마트 시티의 미래 비전을 제시합니다.\n\n1. AI 기반 교통 관리\n2. 친환경 에너지 솔루션\n3. 디지털 트윈 기술 활용' 
  },
  { 
    id: 102, 
    title: 'Neural Network Art', 
    category: 'Motion Graphics', 
    thumbnail: 'https://images.unsplash.com/photo-1618005182384-a83a8bd57fbe?w=800&q=80', 
    url: 'https://www.youtube.com/watch?v=aircAruvnKk', 
    description: 'Generative AI를 활용한 모션 그래픽 실험작.', 
    analysis_status: 'failed', 
    ai_summary: null 
  },
];

/**
 * --------------------------------------------------------------------------
 * [SERVICE] API 통신 계층
 * --------------------------------------------------------------------------
 */
const apiService = {
  login: async (username, password) => {
    try {
      const formData = new URLSearchParams();
      formData.append('username', username);
      formData.append('password', password);
      const res = await fetch(`${CONFIG.API_URL}/auth/login`, {
        method: 'POST',
        headers: { 'Content-Type': 'application/x-www-form-urlencoded' },
        body: formData,
      });
      if (!res.ok) throw new Error('로그인 실패: 아이디/비번을 확인하세요');
      return res.json();
    } catch (error) {
      console.warn("Login failed (backend unreachable?)", error);
      if (username === 'admin' && password === 'password') {
        return { access_token: 'demo_token', token_type: 'bearer' };
      }
      throw error;
    }
  },

  fetchVideos: async () => {
    try {
      const res = await fetch(`${CONFIG.API_URL}/videos/`);
      if (!res.ok) throw new Error('영상 목록 로딩 실패');
      return res.json();
    } catch (error) {
      console.warn("백엔드 연결 실패. Mock 데이터를 사용합니다.", error);
      return MOCK_VIDEOS;
    }
  },

  addVideo: async (video, token) => {
    const res = await fetch(`${CONFIG.API_URL}/videos/`, {
      method: 'POST',
      headers: { 
        'Content-Type': 'application/json',
        'Authorization': `Bearer ${token}`
      },
      body: JSON.stringify(video),
    });
    if (!res.ok) throw new Error('업로드 실패');
    return res.json();
  },

  deleteVideo: async (id, token) => {
    const res = await fetch(`${CONFIG.API_URL}/videos/${id}`, {
      method: 'DELETE',
      headers: { 'Authorization': `Bearer ${token}` },
    });
    if (!res.ok) throw new Error('삭제 실패');
    return true;
  },

  // [NEW] 재분석 요청 API
  retryAnalysis: async (id, token) => {
    const res = await fetch(`${CONFIG.API_URL}/videos/${id}/analyze`, {
      method: 'POST',
      headers: { 'Authorization': `Bearer ${token}` },
    });
    if (!res.ok) throw new Error('재분석 요청 실패');
    return res.json();
  }
};

/**
 * --------------------------------------------------------------------------
 * [STORE] 상태 관리 (Zustand)
 * --------------------------------------------------------------------------
 */
const useStore = create(
  persist(
    (set, get) => ({
      user: null,
      token: null,
      isAuthenticated: false,
      videos: [],
      isLoading: false,
      toast: { show: false, message: '', type: 'info' },

      showToast: (message, type = 'info') => {
        set({ toast: { show: true, message, type } });
        setTimeout(() => set((state) => ({ toast: { ...state.toast, show: false } })), 3000);
      },

      login: async (username, password) => {
        try {
          const data = await apiService.login(username, password);
          set({ token: data.access_token, user: { username }, isAuthenticated: true });
          get().showToast(`환영합니다, ${username}님!`, 'success');
          return true;
        } catch (err) {
          get().showToast(err.message, 'error');
          return false;
        }
      },
      
      logout: () => {
        set({ user: null, token: null, isAuthenticated: false });
        get().showToast('로그아웃 되었습니다.', 'info');
      },
      
      fetchVideos: async () => {
        set({ isLoading: true });
        try {
          const videos = await apiService.fetchVideos();
          set({ videos: videos.sort((a, b) => b.id - a.id), isLoading: false });
        } catch (err) {
          console.error(err);
          set({ isLoading: false });
        }
      },

      addVideo: async (videoData) => {
        const { token, showToast } = get();
        try {
          await apiService.addVideo(videoData, token);
          showToast('프로젝트가 등록되고 분석이 시작되었습니다.', 'success');
          get().fetchVideos();
        } catch (err) {
          showToast('등록에 실패했습니다 (서버 연결 확인 필요).', 'error');
        }
      },

      deleteVideo: async (id) => {
        const { token, videos, showToast } = get();
        try {
          await apiService.deleteVideo(id, token);
          set({ videos: videos.filter(v => v.id !== id) });
          showToast('프로젝트가 삭제되었습니다.', 'success');
        } catch (err) {
          showToast('삭제에 실패했습니다.', 'error');
        }
      },

      // [NEW] 재분석 액션
      retryAnalysis: async (id) => {
        const { token, showToast, videos } = get();
        
        // 로그인 체크
        if (!token) {
          showToast('관리자 로그인이 필요합니다.', 'error');
          return;
        }

        try {
          await apiService.retryAnalysis(id, token);
          // 즉시 UI 상태 업데이트 (낙관적 업데이트)
          set({
            videos: videos.map(v => 
              v.id === id ? { ...v, analysis_status: 'queued' } : v
            )
          });
          showToast('재분석 요청이 대기열에 등록되었습니다.', 'success');
        } catch (err) {
          console.error(err);
          showToast('재분석 요청 실패', 'error');
        }
      }
    }),
    {
      name: 'portfolio-storage',
      storage: createJSONStorage(() => localStorage),
      partialize: (state) => ({ user: state.user, token: state.token, isAuthenticated: state.isAuthenticated }),
    }
  )
);

/**
 * --------------------------------------------------------------------------
 * [COMPONENTS] UI & Features
 * --------------------------------------------------------------------------
 */

// 1. Toast
const Toast = () => {
  const { toast } = useStore();
  if (!toast.show) return null;

  const bgColors = {
    success: 'bg-emerald-500/90 text-white',
    error: 'bg-rose-500/90 text-white',
    info: 'bg-slate-700/90 text-white'
  };

  return (
    <div className={`fixed bottom-8 right-8 z-50 px-6 py-3 rounded-xl shadow-2xl backdrop-blur-md flex items-center gap-3 animate-in slide-in-from-bottom-5 fade-in duration-300 ${bgColors[toast.type]}`}>
      {toast.type === 'success' && <CheckCircle2 size={18} />}
      {toast.type === 'error' && <AlertCircle size={18} />}
      <span className="font-medium text-sm">{toast.message}</span>
    </div>
  );
};

// 2. Modal (Base)
const Modal = ({ isOpen, onClose, title, children, maxWidth = "max-w-md" }) => {
  if (!isOpen) return null;
  return (
    <div className="fixed inset-0 z-50 flex items-center justify-center bg-slate-950/80 backdrop-blur-sm p-4 animate-in fade-in duration-200">
      <div className={`bg-slate-900 border border-slate-800 w-full ${maxWidth} rounded-2xl shadow-2xl overflow-hidden animate-in zoom-in-95 duration-200`}>
        <div className="flex justify-between items-center p-4 border-b border-slate-800 bg-slate-900/50">
          <h3 className="font-bold text-lg text-slate-100 flex items-center gap-2">{title}</h3>
          <button onClick={onClose} className="text-slate-500 hover:text-white transition-colors"><X size={20}/></button>
        </div>
        <div className="p-0">
          {children}
        </div>
      </div>
    </div>
  );
};

// 3. Video Player Modal
const VideoPlayerModal = ({ video, isOpen, onClose }) => {
  if (!isOpen || !video) return null;
  
  const getEmbedUrl = (url) => {
    const regExp = /^.*(youtu.be\/|v\/|u\/\w\/|embed\/|watch\?v=|&v=)([^#&?]*).*/;
    const match = url.match(regExp);
    const videoId = (match && match[2].length === 11) ? match[2] : null;
    return videoId ? `https://www.youtube.com/embed/${videoId}?autoplay=1` : null;
  };

  const embedUrl = getEmbedUrl(video.url);

  return (
    <Modal isOpen={isOpen} onClose={onClose} title={video.title} maxWidth="max-w-4xl">
      <div className="aspect-video bg-black w-full">
        {embedUrl ? (
          <iframe 
            src={embedUrl} 
            className="w-full h-full" 
            allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" 
            allowFullScreen
          />
        ) : (
          <div className="flex items-center justify-center h-full text-slate-500">
            지원되지 않는 동영상 URL입니다.
          </div>
        )}
      </div>
      <div className="p-6 bg-slate-900">
        <h4 className="font-bold text-white mb-2">{video.title}</h4>
        <p className="text-slate-400 text-sm">{video.description}</p>
      </div>
    </Modal>
  );
};

// 4. AI Summary Modal
const AISummaryModal = ({ video, isOpen, onClose }) => {
  if (!isOpen || !video) return null;
  return (
    <Modal isOpen={isOpen} onClose={onClose} title="AI Analysis Report" maxWidth="max-w-2xl">
      <div className="p-6 max-h-[70vh] overflow-y-auto">
        <div className="flex items-center gap-3 mb-6 p-4 bg-indigo-500/10 border border-indigo-500/20 rounded-xl">
          <div className="p-2 bg-indigo-500 rounded-lg">
            <Sparkles size={20} className="text-white" />
          </div>
          <div>
            <h5 className="font-bold text-indigo-300 text-sm">AI Video Analysis</h5>
            <p className="text-xs text-indigo-400/70">Vision Model(MiniCPM-V)과 LLM(Qwen3)이 영상을 분석한 결과입니다.</p>
          </div>
        </div>
        
        <div className="prose prose-invert prose-sm max-w-none">
          <div className="whitespace-pre-wrap leading-relaxed text-slate-300">
            {video.ai_summary || "요약 정보가 없습니다."}
          </div>
        </div>
      </div>
    </Modal>
  );
};

// 5. Video Card (Retry 버튼 추가)
const VideoCard = ({ video, isAdmin, onDelete, onPlay, onShowSummary, onRetry }) => {
  const statusConfig = {
    pending: { color: 'bg-slate-700 text-slate-300', icon: null, text: '대기중' },
    queued: { color: 'bg-yellow-500/20 text-yellow-400 border-yellow-500/30', icon: <Loader2 size={12} className="animate-spin"/>, text: '분석 대기' },
    processing: { color: 'bg-blue-500/20 text-blue-400 border-blue-500/30', icon: <Loader2 size={12} className="animate-spin"/>, text: 'AI 분석중' },
    completed: { color: 'bg-indigo-500/20 text-indigo-400 border-indigo-500/30', icon: <Sparkles size={12}/>, text: '분석 완료' },
    failed: { color: 'bg-red-500/20 text-red-400 border-red-500/30', icon: <AlertCircle size={12}/>, text: '분석 실패' },
  };

  const status = statusConfig[video.analysis_status] || statusConfig.pending;

  return (
    <div className="group relative bg-slate-900 rounded-2xl overflow-hidden border border-slate-800 hover:border-indigo-500/50 transition-all duration-300 hover:shadow-2xl hover:shadow-indigo-500/10 hover:-translate-y-1 flex flex-col h-full">
      <div className="aspect-video relative overflow-hidden bg-slate-950">
        <img 
          src={video.thumbnail} 
          alt={video.title} 
          className="w-full h-full object-cover transition-transform duration-700 group-hover:scale-105 opacity-80 group-hover:opacity-100"
          onError={(e) => { e.target.src = 'https://placehold.co/640x360/1e293b/cbd5e1.png?text=No+Preview'; }}
        />
        <div className="absolute inset-0 bg-black/40 opacity-0 group-hover:opacity-100 transition-opacity duration-300 flex items-center justify-center backdrop-blur-[2px]">
          <button 
            onClick={() => onPlay(video)}
            className="transform scale-90 group-hover:scale-100 transition-transform duration-300 w-16 h-16 rounded-full bg-white/10 backdrop-blur-md border border-white/20 flex items-center justify-center text-white hover:bg-white hover:text-black hover:border-white shadow-xl"
          >
            <Play fill="currentColor" size={24} className="ml-1" />
          </button>
        </div>
        <span className="absolute top-3 left-3 bg-black/60 backdrop-blur-md text-[10px] font-bold uppercase tracking-wider text-white px-2.5 py-1 rounded-md border border-white/10">
          {video.category}
        </span>
      </div>

      <div className="p-5 flex flex-col flex-1">
        <div className="mb-auto">
          <div className="flex justify-between items-start gap-4 mb-3">
            <h3 className="font-bold text-lg text-slate-100 line-clamp-1 group-hover:text-indigo-400 transition-colors">
              {video.title}
            </h3>
          </div>
          <p className="text-slate-400 text-sm line-clamp-2 leading-relaxed mb-4">
            {video.description}
          </p>
        </div>

        <div className="pt-4 mt-2 border-t border-slate-800 flex items-center justify-between">
          <div className={`flex items-center gap-1.5 px-2.5 py-1 rounded-full text-xs font-medium border ${status.color.includes('border') ? status.color : 'border-transparent ' + status.color}`}>
            {status.icon}
            <span>{status.text}</span>
          </div>

          <div className="flex items-center gap-2">
            {/* [NEW] 재시도 버튼 - 관리자만 보이도록 수정 */}
            {isAdmin && video.analysis_status === 'failed' && (
              <button 
                onClick={() => onRetry(video.id)}
                className="p-2 text-slate-400 hover:text-yellow-400 hover:bg-yellow-500/10 rounded-lg transition-colors"
                title="분석 재시도"
              >
                <RefreshCw size={18} />
              </button>
            )}

            {video.analysis_status === 'completed' && (
              <button 
                onClick={() => onShowSummary(video)}
                className="p-2 text-slate-400 hover:text-indigo-400 hover:bg-indigo-500/10 rounded-lg transition-colors"
                title="AI 요약 보기"
              >
                <FileText size={18} />
              </button>
            )}
            
            {isAdmin && (
              <button 
                onClick={() => onDelete(video.id)}
                className="p-2 text-slate-400 hover:text-red-400 hover:bg-red-500/10 rounded-lg transition-colors"
                title="삭제"
              >
                <Trash2 size={18} />
              </button>
            )}
          </div>
        </div>
      </div>
    </div>
  );
};

/**
 * --------------------------------------------------------------------------
 * [APP] Main Application
 * --------------------------------------------------------------------------
 */
export default function App() {
  const { videos, isAuthenticated, user, login, logout, fetchVideos, deleteVideo, addVideo, retryAnalysis, isLoading } = useStore();
  
  const [search, setSearch] = useState('');
  const [filterCategory, setFilterCategory] = useState('All');
  const [activeVideo, setActiveVideo] = useState(null);
  const [summaryVideo, setSummaryVideo] = useState(null);
  const [isLoginOpen, setIsLoginOpen] = useState(false);
  const [isUploadOpen, setIsUploadOpen] = useState(false);
  
  const [loginForm, setLoginForm] = useState({ username: '', password: '' });
  const [uploadForm, setUploadForm] = useState({ title: '', category: '', description: '', url: '', thumbnail: '' });

  useEffect(() => {
    fetchVideos();
    const interval = setInterval(fetchVideos, 10000);
    return () => clearInterval(interval);
  }, []);

  const categories = useMemo(() => ['All', ...new Set(videos.map(v => v.category))], [videos]);
  const filteredVideos = useMemo(() => {
    return videos.filter(v => {
      const matchesSearch = v.title.toLowerCase().includes(search.toLowerCase()) || 
                            v.description.toLowerCase().includes(search.toLowerCase());
      const matchesCategory = filterCategory === 'All' || v.category === filterCategory;
      return matchesSearch && matchesCategory;
    });
  }, [videos, search, filterCategory]);

  const handleLogin = async (e) => {
    e.preventDefault();
    if (await login(loginForm.username, loginForm.password)) {
      setLoginForm({ username: '', password: '' });
      setIsLoginOpen(false);
    }
  };

  const handleUpload = async (e) => {
    e.preventDefault();
    await addVideo(uploadForm);
    setUploadForm({ title: '', category: '', description: '', url: '', thumbnail: '' });
    setIsUploadOpen(false);
  };

  return (
    <div className="min-h-screen bg-slate-950 text-slate-200 font-sans selection:bg-indigo-500/30">
      <Toast />
      
      <nav className="fixed top-0 z-40 w-full backdrop-blur-xl border-b border-white/5 bg-slate-950/80">
        <div className="max-w-7xl mx-auto px-4 sm:px-6 lg:px-8 h-16 flex items-center justify-between">
          <div className="flex items-center gap-2 cursor-pointer" onClick={() => window.scrollTo(0,0)}>
            <div className="w-8 h-8 bg-gradient-to-br from-indigo-500 to-purple-600 rounded-lg flex items-center justify-center shadow-lg shadow-indigo-500/20">
              <Play fill="white" size={14} className="ml-0.5 text-white" />
            </div>
            <span className="font-bold text-xl tracking-tight text-white">Portfolio.io</span>
          </div>

          <div className="flex items-center gap-3">
            {isAuthenticated ? (
              <>
                <span className="hidden sm:block text-xs font-medium px-3 py-1.5 rounded-full bg-slate-800 border border-slate-700 text-slate-300">
                  <span className="text-indigo-400">●</span> {user?.username}
                </span>
                <button onClick={() => setIsUploadOpen(true)} className="flex items-center gap-2 bg-indigo-600 hover:bg-indigo-700 text-white px-4 py-2 rounded-lg text-sm font-medium transition-all shadow-lg shadow-indigo-500/20">
                  <Plus size={16} /> <span className="hidden sm:inline">New Project</span>
                </button>
                <button onClick={logout} className="p-2 text-slate-400 hover:text-white hover:bg-white/5 rounded-lg transition-colors">
                  <LogOut size={18} />
                </button>
              </>
            ) : (
              <button onClick={() => setIsLoginOpen(true)} className="flex items-center gap-2 text-sm font-medium text-slate-400 hover:text-white transition-colors px-3 py-2 rounded-lg hover:bg-white/5">
                <Lock size={16} /> Admin
              </button>
            )}
          </div>
        </div>
      </nav>

      <div className="relative pt-32 pb-20 sm:pt-40 sm:pb-24 overflow-hidden">
        <div className="absolute top-0 left-1/2 -translate-x-1/2 w-[1000px] h-[500px] bg-indigo-600/20 rounded-full blur-[120px] pointer-events-none opacity-50 mix-blend-screen" />
        
        <div className="relative max-w-7xl mx-auto px-4 sm:px-6 lg:px-8 text-center z-10">
          <div className="inline-flex items-center gap-2 px-3 py-1 rounded-full bg-slate-800/50 border border-slate-700/50 backdrop-blur-md mb-8">
            <span className="flex h-2 w-2 rounded-full bg-emerald-400 animate-pulse"></span>
            <span className="text-xs font-medium text-slate-300">AI Powered Analysis Available</span>
          </div>
          
          <h1 className="text-5xl sm:text-7xl font-extrabold text-transparent bg-clip-text bg-gradient-to-b from-white via-white to-slate-500 mb-6 tracking-tight leading-tight">
            Creative Motion <br /> & Visual Storytelling
          </h1>
          <p className="text-lg sm:text-xl text-slate-400 max-w-2xl mx-auto mb-10 leading-relaxed font-light">
            브랜드의 가치를 시각적 언어로 통역합니다.<br className="hidden sm:block"/>
            AI 기술을 활용한 심층적인 영상 분석과 인사이트를 제공합니다.
          </p>
          
          <div className="max-w-2xl mx-auto bg-slate-900/80 backdrop-blur-xl p-2 rounded-2xl border border-white/10 shadow-2xl flex flex-col sm:flex-row gap-2">
            <div className="relative flex-1 group">
              <Search className="absolute left-3 top-1/2 -translate-y-1/2 text-slate-500 group-focus-within:text-indigo-400 transition-colors" size={18} />
              <input 
                type="text" 
                placeholder="Search projects..." 
                className="w-full bg-transparent border-none text-white placeholder-slate-500 pl-10 pr-4 py-2.5 focus:outline-none text-sm"
                value={search}
                onChange={(e) => setSearch(e.target.value)}
              />
            </div>
            <div className="flex gap-1 overflow-x-auto no-scrollbar pb-1 sm:pb-0 pl-2 sm:pl-0 sm:border-l border-white/10">
              {categories.map(cat => (
                <button
                  key={cat}
                  onClick={() => setFilterCategory(cat)}
                  className={`px-3 py-1.5 rounded-lg text-xs font-medium whitespace-nowrap transition-all ${
                    filterCategory === cat 
                      ? 'bg-slate-700 text-white shadow-sm' 
                      : 'text-slate-400 hover:text-white hover:bg-slate-800'
                  }`}
                >
                  {cat}
                </button>
              ))}
            </div>
          </div>
        </div>
      </div>

      <main className="max-w-7xl mx-auto px-4 sm:px-6 lg:px-8 pb-32">
        {isLoading && videos.length === 0 ? (
          <div className="grid grid-cols-1 sm:grid-cols-2 lg:grid-cols-3 gap-8">
            {[1,2,3].map(i => (
              <div key={i} className="bg-slate-900 rounded-2xl h-[400px] animate-pulse border border-slate-800">
                <div className="h-[220px] bg-slate-800 rounded-t-2xl"/>
                <div className="p-5 space-y-3">
                  <div className="h-6 bg-slate-800 rounded w-3/4"/>
                  <div className="h-4 bg-slate-800 rounded w-full"/>
                  <div className="h-4 bg-slate-800 rounded w-2/3"/>
                </div>
              </div>
            ))}
          </div>
        ) : filteredVideos.length > 0 ? (
          <div className="grid grid-cols-1 sm:grid-cols-2 lg:grid-cols-3 gap-8">
            {filteredVideos.map((video) => (
              <VideoCard 
                key={video.id} 
                video={video} 
                isAdmin={isAuthenticated}
                onDelete={deleteVideo}
                onPlay={setActiveVideo}
                onShowSummary={setSummaryVideo}
                onRetry={retryAnalysis}
              />
            ))}
          </div>
        ) : (
          <div className="text-center py-32 border border-dashed border-slate-800 rounded-3xl bg-slate-900/30">
            <div className="inline-flex p-4 rounded-full bg-slate-800 mb-4 text-slate-500">
              <Search size={24} />
            </div>
            <p className="text-slate-400 text-lg font-medium">No projects found.</p>
            <button onClick={() => {setSearch(''); setFilterCategory('All')}} className="mt-2 text-indigo-400 hover:text-indigo-300 text-sm">
              Clear all filters
            </button>
          </div>
        )}
      </main>

      <footer className="border-t border-white/5 bg-slate-950 py-12">
        <div className="max-w-7xl mx-auto px-4 text-center">
          <p className="text-slate-500 text-sm">© 2025 Visual Portfolio. Built with FastAPI & React.</p>
        </div>
      </footer>

      <VideoPlayerModal 
        video={activeVideo} 
        isOpen={!!activeVideo} 
        onClose={() => setActiveVideo(null)} 
      />
      
      <AISummaryModal 
        video={summaryVideo} 
        isOpen={!!summaryVideo} 
        onClose={() => setSummaryVideo(null)} 
      />

      <Modal isOpen={isLoginOpen} onClose={() => setIsLoginOpen(false)} title="관리자 로그인">
        <form onSubmit={handleLogin} className="space-y-4 p-6">
          <div className="bg-indigo-500/10 border border-indigo-500/20 rounded-lg p-3 text-center">
             <span className="text-xs text-indigo-300">admin / password</span>
          </div>
          <div>
            <label className="text-xs font-medium text-slate-400 mb-1.5 block">Username</label>
            <input type="text" className="w-full bg-slate-950 border border-slate-700 rounded-lg px-4 py-2.5 text-white focus:ring-2 focus:ring-indigo-500 focus:border-transparent outline-none transition-all" value={loginForm.username} onChange={e => setLoginForm({...loginForm, username: e.target.value})} />
          </div>
          <div>
            <label className="text-xs font-medium text-slate-400 mb-1.5 block">Password</label>
            <input type="password" className="w-full bg-slate-950 border border-slate-700 rounded-lg px-4 py-2.5 text-white focus:ring-2 focus:ring-indigo-500 focus:border-transparent outline-none transition-all" value={loginForm.password} onChange={e => setLoginForm({...loginForm, password: e.target.value})} />
          </div>
          <button className="w-full bg-indigo-600 hover:bg-indigo-700 text-white font-medium py-2.5 rounded-lg transition-all shadow-lg shadow-indigo-500/20">로그인</button>
        </form>
      </Modal>

      <Modal isOpen={isUploadOpen} onClose={() => setIsUploadOpen(false)} title="새 프로젝트 업로드">
        <form onSubmit={handleUpload} className="space-y-4 p-6">
          <div>
            <label className="text-xs font-medium text-slate-400 mb-1.5 block">Project Title</label>
            <input required type="text" className="w-full bg-slate-950 border border-slate-700 rounded-lg px-4 py-2.5 text-white focus:ring-2 focus:ring-indigo-500 outline-none" value={uploadForm.title} onChange={e => setUploadForm({...uploadForm, title: e.target.value})} />
          </div>
          <div className="grid grid-cols-2 gap-4">
             <div>
              <label className="text-xs font-medium text-slate-400 mb-1.5 block">Category</label>
              <input required type="text" placeholder="Commercial" className="w-full bg-slate-950 border border-slate-700 rounded-lg px-4 py-2.5 text-white focus:ring-2 focus:ring-indigo-500 outline-none" value={uploadForm.category} onChange={e => setUploadForm({...uploadForm, category: e.target.value})} />
             </div>
             <div>
              <label className="text-xs font-medium text-slate-400 mb-1.5 block">Video URL (YouTube)</label>
              <input type="text" placeholder="https://youtube.com/..." className="w-full bg-slate-950 border border-slate-700 rounded-lg px-4 py-2.5 text-white focus:ring-2 focus:ring-indigo-500 outline-none" value={uploadForm.url} onChange={e => setUploadForm({...uploadForm, url: e.target.value})} />
             </div>
          </div>
          <div>
            <label className="text-xs font-medium text-slate-400 mb-1.5 block">Description</label>
            <textarea className="w-full bg-slate-950 border border-slate-700 rounded-lg px-4 py-2.5 text-white focus:ring-2 focus:ring-indigo-500 outline-none h-24 resize-none" value={uploadForm.description} onChange={e => setUploadForm({...uploadForm, description: e.target.value})} />
          </div>
          <button className="w-full bg-indigo-600 hover:bg-indigo-700 text-white font-medium py-2.5 rounded-lg transition-all shadow-lg shadow-indigo-500/20">프로젝트 등록</button>
        </form>
      </Modal>
    </div>
  );
}